# Model Prototyping
Geoff Pidcock | PacifImpact | Sept 2020

## Scope
- Review prior examples of "Nowcasting"
  - what algorithms
  - how were they evaluated
  - were there any gold standards
- Apply method to toy data example

# Review

#### Macroeconomic Nowcasting with Kalman Filtering, Jason Yip (Medium)
[link](https://towardsdatascience.com/macroeconomic-nowcasting-with-kalman-filtering-557926dbc737)
- References `Macroeconomic Nowcasting and Forecasting with Big Data` (Brandyn Bok et al, 2018) [link](https://www.annualreviews.org/doi/abs/10.1146/annurev-economics-080217-053214) (behind a paywall)
   - [version that's open](https://www.newyorkfed.org/medialibrary/media/research/staff_reports/sr830.pdf)
   - 55 citations
- 

#### Analysing port and shipping operations using big data, Bonham et al (2018)
[link](https://datasciencecampus.ons.gov.uk/wp-content/uploads/sites/10/2020/04/Report_Analysing_port_and_shipping_operations_using_big_data_June2018.pdf)
